## Building PyTorch from Source on Local Machine with CUDA Support
Compiling from source to **be able to run PyTorch with GPU support and compile PyTorch extensions with CUDA support**

Systems tested:
* [TBD system] with CUDA 10.0 and cuDNN 7.3.0 ([CUDA setup](https://docs.nvidia.com/cuda/cuda-installation-guide-mac-os-x/index.html) and [cuDNN setup](https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html)) and Python 3.6 (non-conda install)

In [ ]:
# What pytorch do we have right now?
import sys
print(sys.prefix)
! {sys.prefix}/bin/pip freeze | grep torch

Please have done the following if on macOS (tested with 10.13):
1. Pip installed all packages from `requirements.txt` (`pip install -r requirements.txt`)
2. Make sure using `clang` and `clang++` from Apple through XCode.
3. Homebrew install of `libbind`:  `brew install libbind`
4.  NVIDIA CUDA Toolkit setup ([NVIDIA CUDA Toolkit Docs](https://docs.nvidia.com/cuda/index.html))
  * NEED NVIDIA CUDA >= 9.2 (testing here with 10.0)


These instructions are based on 
* https://github.com/QuantScientist/Deep-Learning-Boot-Camp/blob/master/day02-PyTORCH-and-PyCUDA/PyTorch/build_torch.sh
* http://www.cs.rochester.edu/u/kautz/Installing-Pytorch-Cuda-on-Macbook.html
* https://github.com/pytorch/pytorch/blob/master/.jenkins/pytorch/macos-build.sh
* https://github.com/apache/incubator-mxnet/issues/9217#issuecomment-354982838

In [ ]:
! git clone --recursive https://github.com/pytorch/pytorch.git

In [ ]:
! export PATH=/usr/local/cuda/bin:$PATH &&\
    export LD_LIBRARY_PATH=/usr/local/cuda/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/usr/local/cuda/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-10.0 &&\
    export CUDNN_LIB_DIR=/usr/local/cuda/lib &&\
    export CUDA_HOST_COMPILER=clang &&\
    export USE_CUDA=1 &&\
    export USE_NNPACK=1 &&\
    export CC=clang &&\
    export CXX=clang++ &&\
    cd pytorch &&\
    pip3 uninstall torch --yes &&\
    python3 setup.py clean &&\
    TORCH_CUDA_ARCH_LIST="5.2" python3 setup.py install

In [ ]:
# # To redo above, remove the folder that was git cloned
# ! rm -fr pytorch

In [ ]:
import torch
torch.__version__

It might be prudent to build a wheel for this build.  To do so, on the command line or in a notebook cell, in the `pytorch` folder: [wip command]

`python3 setup.py clean && PATH=/Developer/NVIDIA/CUDA-10.0/bin:$PATH LD_LIBRARY_PATH=//Developer/NVIDIA/CUDA-10.0/lib:$LD_LIBRARY_PATH CUDA_BIN_PATH=/Developer/NVIDIA/CUDA-10.0/bin CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-10.0 CUDNN_LIB_DIR=//Developer/NVIDIA/CUDA-10.0/lib USE_CUDA=1 CC=clang CXX=clang++ CUDA_HOME=/Developer/NVIDIA/CUDA-10.0/ USE_NNPACK=0 LDFLAGS="-Wl,-no_compact_unwind" python3 setup.py sdist bdist_wheel`

Then the .whl file will be in the `bdist` folder.

Building on an NVIDIA PX2 Drive (CUDA 9.0, Python 3.5, Ubuntu 16.04):

* Note, need a swap file - instructions here https://support.rackspace.com/how-to/create-a-linux-swap-file/ (created 3GB swap file).
* This will build a wheel under `bdist`

In [ ]:
! export PATH=/usr/local/cuda/bin:$PATH &&\
    export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH &&\
    export LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/usr/local/cuda/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda/ &&\
    export CUDNN_LIB_DIR=/usr/local/cuda/lib64 &&\
    export CUDA_HOST_COMPILER=cc &&\
    export USE_CUDA=1 &&\
    export USE_NNPACK=1 &&\
    export CC=cc &&\
    export CXX=c++ &&\
    cd pytorch &&\
    pip3 uninstall torch --yes &&\
    python3 setup.py clean &&\
    TORCH_CUDA_ARCH_LIST="3.5 5.2 6.0 6.1+PTX" TORCH_NVCC_FLAGS="-Xfatbin -compress-all" python3 setup.py install